In [1]:
import requests
import json
import urllib
from urllib.parse import urlparse, urljoin, quote
from requests_html import HTML, HTMLSession
import undetected_chromedriver as uc
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from datetime import date
import time
import pandas as pd
import numpy as np
import whois
import datetime
import warnings
import networkx as nx

# URL pagerank

In [43]:
def get_links(url):
    driver = webdriver.Chrome()
    warnings.filterwarnings("ignore", category=DeprecationWarning)
    driver.get(url)
    link = driver.find_elements(By.TAG_NAME, 'a')
    links = []
    domains = []
    try:
        for i in link:
            href = i.get_attribute('href')
            if href is not None and href != 'javascript:void(0)' and href.startswith('http'):
                links.append(href)
                a= urlparse(href)
                base_url = f"{a.scheme}://{a.netloc}"
                domains.append(base_url)
    except:
        pass
    links.extend(list(set(domains)))
    driver.close()
    return links

In [10]:
def crawl_urls(urls, depth=1, max_depth=3):
    # Base case: reached the maximum depth or no more URLs to crawl
    if depth > max_depth or not urls:
        return {}
    
    # Create an empty dictionary to store the results
    url_dict = {}
    
    # Iterate over the URLs and add edges based on links between them
    for url in urls:
        # Retrieve the links for the current URL
        links = get_links(url)

        # Process the linked URLs and add them to the dictionary
        url_dict[url] = links

        # Recursively crawl the linked URLs
        nested_dict = crawl_urls(links, depth=depth+1, max_depth=max_depth)
        
        # Merge the nested dictionary with the main dictionary
        url_dict.update(nested_dict)
        
    return url_dict

1 level PR

In [75]:
# Create empty lists for source and destination URLs
src_urls = []
dst_urls = []

# Iterate over the dictionary and extract source and destination URLs
for src, dst_list in dict_.items():
    # Append the source URL multiple times based on the number of destination URLs
    src_urls.extend([src] * len(dst_list))
    # Append the destination URLs
    dst_urls.extend(dst_list)
# Create a DataFrame from the lists
pr = pd.DataFrame({'src': src_urls, 'dst': dst_urls}) 
# pr = pd.DataFrame.from_dict(dict_, orient='columns').T.reset_index()
# pr.columns = ['src', 'dsc']
pr

,src,dst
0,https://www.rightmove.co.uk/property-to-rent/L...,https://www.rightmove.co.uk/this-site/cookies....
1,https://www.rightmove.co.uk/property-to-rent/L...,https://onetrust.com/poweredbyonetrust
2,https://www.rightmove.co.uk/property-to-rent/L...,https://www.rightmove.co.uk/
3,https://www.rightmove.co.uk/property-to-rent/L...,https://www.rightmove.co.uk/
4,https://www.rightmove.co.uk/property-to-rent/L...,https://www.rightmove.co.uk/property-to-rent/L...
...,...,...
1291858,https://www.zoopla.co.uk/house-prices/central-...,https://pro.zoopla.co.uk
1291859,https://www.zoopla.co.uk/house-prices/central-...,https://uk.pinterest.com
1291860,https://www.zoopla.co.uk/house-prices/central-...,https://careers.zoopla.co.uk
1291861,https://www.zoopla.co.uk/house-prices/central-...,https://www.altosoftware.co.uk


Create link between URL and domain before building the graph for PR calculation

In [113]:
src1 = list(pr['dst'].unique())
src2= list(pr['src'].unique())

In [95]:
src = src1.extend(src2)

In [115]:
# src = list(set(np.concatenate((pr['dst'].unique(), pr['src'].unique()), axis=0)))

In [117]:
src = list(set(np.concatenate((pr['dst'].unique(), pr['src'].unique()), axis=0)))
dst = []
for i in src:
    a= urlparse(i)
    base_url = f"{a.scheme}://{a.netloc}"
    dst.append(base_url)

In [120]:
df1 = pd.DataFrame({'src': src, 'dst': dst})
df2 = pd.DataFrame({'src': dst, 'dst': src})

In [122]:
df3 = pd.concat([df1, df2])
df3

,src,dst
0,https://nethouseprices.com/agent-details/11273ST2,https://nethouseprices.com
1,https://goadsby.com/knowledge-hub/sub-category...,https://goadsby.com
2,https://www.rightmove.co.uk/properties/1335820...,https://www.rightmove.co.uk
3,https://www.zoopla.co.uk/myaccount/hidden-list...,https://www.zoopla.co.uk
4,https://www.bing.com/aclick?ld=e80hHDQySSW7OAt...,https://www.bing.com
...,...,...
333974,https://www.rightmove.co.uk,https://www.rightmove.co.uk/properties/1342284...
333975,https://www.rightmove.co.uk,https://www.rightmove.co.uk/overseas-property/...
333976,https://www.primelocation.com,https://www.primelocation.com/myaccount/favour...
333977,https://www.mcclellandsalter.co.uk,https://www.mcclellandsalter.co.uk/about-us


In [123]:
pr2 = pd.concat([pr, df3])

Create graph for PR calculation, using NetworkX

In [124]:
G2 = nx.from_pandas_edgelist(pr2, source='src', target='dst', create_using=nx.DiGraph)
pagerank2 = nx.pagerank(G2)
pagerank2

{'https://www.rightmove.co.uk/property-to-rent/London/1-bed-houses.html': 1.8936997477556171e-06,
 'https://www.rightmove.co.uk/this-site/cookies.html': 5.904326165893544e-06,
 'https://onetrust.com/poweredbyonetrust': 2.0000354389897553e-05,
 'https://www.rightmove.co.uk/': 6.381994921647452e-06,
 'https://www.rightmove.co.uk/property-to-rent/London/1-bed-houses.html#': 1.8644618447572699e-06,
 'https://www.rightmove.co.uk/property-for-sale.html': 6.084555478232327e-06,
 'https://www.rightmove.co.uk/new-homes-for-sale.html': 6.084555478232327e-06,
 'https://where.rightmove.co.uk/': 2.0162965821511662e-05,
 'https://www.rightmove.co.uk/property-valuation.html': 6.084555478232327e-06,
 'https://www.rightmove.co.uk/guides/landlord/investor-newsletter/': 6.084555478232327e-06,
 'https://www.rightmove.co.uk/mortgages': 5.46435611104554e-06,
 'https://www.rightmove.co.uk/property-to-rent.html': 6.09894702633821e-06,
 'https://www.rightmove.co.uk/student-accommodation.html': 6.08455547823232

In [35]:
data = pd.read_excel('processed_data.xlsx')

In [125]:
url_list = list(data['LP_link'].unique())
pagerank_list = [pagerank2.get(url, 0) for url in url_list]

# Create DataFrame with URL and corresponding PageRank values
pagerank_mapping = pd.DataFrame({'URL': url_list, 'PageRank': pagerank_list})

# Print the DataFrame
print(pagerank_mapping)

                                                    URL  PageRank
0     https://www.rightmove.co.uk/property-to-rent/L...  0.000002
1     https://www.zoopla.co.uk/to-rent/houses/1-bedr...  0.000002
2     https://www.zoopla.co.uk/to-rent/houses/1-bedr...  0.000002
3     https://www.onthemarket.com/to-rent/1-bed-hous...  0.000002
4     https://www.gumtree.com/flats-houses/property-...  0.000002
...                                                 ...       ...
5584                https://www.zoopla.co.uk/valuation/  0.000007
5585  https://www.zoopla.co.uk/discover/property-new...  0.000002
5586      https://www.zoopla.co.uk/house-prices/street/  0.000002
5587  https://www.zoopla.co.uk/house-prices/london/e...  0.000002
5588  https://www.zoopla.co.uk/house-prices/central-...  0.000002

[5589 rows x 2 columns]


In [127]:
pagerank_mapping.to_csv('pagerank_mapping.csv')

# API domain pagerank from openpagerank

In [33]:
api_key = 'wkgogoscc0ccog8ocwwg00s0kk0w8w4sg0cwkgww'
url = 'https://openpagerank.com/api/v1.0/getPageRank'

headers = {'API-OPR': api_key}
params = {'domains[]': ['google.com', 'apple.com', 'rightmove.co.uk/property-to-rent/London/1-bed-houses.html']}

response = requests.get(url, headers=headers, params=params)

In [34]:
response.json()

{'status_code': 200,
 'response': [{'status_code': 200,
   'error': '',
   'page_rank_integer': 10,
   'page_rank_decimal': 10,
   'rank': '3',
   'domain': 'google.com'},
  {'status_code': 200,
   'error': '',
   'page_rank_integer': 8,
   'page_rank_decimal': 7.92,
   'rank': '70',
   'domain': 'apple.com'},
  {'status_code': 400,
   'error': 'Invalid Domain',
   'page_rank_integer': '',
   'page_rank_decimal': '',
   'rank': '',
   'domain': 'rightmove.co.uk/property-to-rent/london/1-bed-houses.html'}],
 'last_updated': '17th Mar 2023'}